# Loading and running multiple experiments

The `Experiment` class provides a simple way to run multiple experiments in a batch.  To do so we can create multiple instances of `Experiment`, each with a different set of inputs for the model.  These are then executed in a loop.

A method to implement this in `streamlit` is to upload a Comma Separated Value (.CSV) file containing a list of experiments to the web app.  This can be stored internally as a `pandas.Dataframe` and displayed using a `streamlit` widget such as `st.table`.  A user can then edit experiment files locally on their own machine (for example, using a spreadsheet software or using CSV viewer extensions for Jupyter-Lab or Visual Studio Code) and upload, inspect, and run, and view results in the app.

## Formatting experiment files

In the format used here each row represents an experiment. The first column is a unique numeric identifier, the second column a name given to the experiment, and following n columns represent the optional input variables that can be passed to an `Experiment`.

Note that the method described here relies on the names of these columns matching the input parameters to `Experiment`.

> But note that columns do not need to be in the same order as `Experiment` arguments and they do not need to be exhaustive.  A selection works fine.

For example, in the urgent care call centre we will include 3 columns with the names:

* n_operators
* n_nurses
* chance_callback

The function `create_example_csv()` creates such a file containing four experiments that vary these paramters.

In [1]:
import pandas as pd

In [4]:
def create_example_csv(filename='example_experiments.csv'):
    '''
    Create an example CSV file to use in tutorial.
    This creates 4 experiments that varys
    n_operators, n_nurses, and chance_callback

    Params:
    ------
    filename: str, optional (default='example_experiments.csv')
        The name and path to the CSV file.
    '''
    # each column is defined as a seperate list
    names = ['base', 'op+1', 'nurse+1', 'high_acuity']
    operators = [13, 14, 13, 13]
    nurses = [9, 9, 10, 9]
    chance_callback = [0.4, 0.4, 0.4, 0.55]

    # empty dataframe
    df_experiments = pd.DataFrame()

    # create new columns from lists
    df_experiments['experiment'] = names
    df_experiments['n_operators'] = operators
    df_experiments['n_nurses'] = nurses
    df_experiments['chance_callback'] = chance_callback

    df_experiments.to_csv(filename, index_label='id')

In [5]:
create_example_csv()

## Uploading a file to a web app

`streamlit` provides the `st.file_uploader` function to easily upload the file to the app.  This is displayed as a button in the that prompts the user with a file open dialog window when clicked.  The user then selected the CSV file and it is uploaded and displayed.  The status of the file upload (True or False) can also be assigned to a variable.  The following code can be used to do so.

```python
uploaded_file = st.file_uploader("Choose a file")
if uploaded_file is not None:
    # assumes CSV format: read into dataframe.
    df_experiments = pd.read_csv(uploaded_file, index_col=0)
    st.write('**Loaded Experiments**')
    st.table(df_experiments)
```

## Converting the upload to instances of `Experiment`

Once the upload is complete, the code above displays to the user and stores as a `pd.Dataframe` in the `df_experiments` variable.  To convert the rows to `Experiment` objects is a two step process. 

1. We cast the `Dataframe` to a nested python dictionary. Each key in the dictionary is the name of an experiment. The value is another dictionary where the key/value pairs are columns and their values.  
2. We loop through the dictionary entries and pass the parameters to a new instance of the `Experiment`` class.

The function `create_experiments` implements both of these steps. The function returns a new dictionary where the key value pairs are the experiment name string, and an instance of `Experiment` 

In [6]:
from model import Experiment

In [7]:
def create_experiments(df_experiments):
    '''
    Returns dictionary of Experiment objects based on contents of a dataframe

    Params:
    ------
    df_experiments: pandas.DataFrame
        Dataframe of experiments. First two columns are id, name followed by 
        variable names.  No fixed width

    Returns:
    --------
    dict
    '''
    experiments = {}
    
    # experiment input parameter dictionary
    exp_dict = df_experiments[df_experiments.columns[1:]].T.to_dict()
    # names of experiments
    exp_names = df_experiments[df_experiments.columns[0]].T.to_list()
    
    # loop through params and create Experiment objects.
    for name, params in zip(exp_names, exp_dict.values()):
        experiments[name] = Experiment(**params)
    
    return experiments

In [8]:
# test of the function

# assume code is run in same directory as example csv file
df_experiment = pd.read_csv('example_experiments.csv')

experiments_to_run = create_experiments(df_experiment)

print(type(experiments_to_run))

print(experiments_to_run['base'].n_nurses)

TypeError: __init__() got an unexpected keyword argument 'experiment'

In [13]:

params = {'mean_iat':0.8, 'n_operators': 14}

args = Experiment(**params)

In [14]:
args.arrival_dist.mean

0.8

In [16]:
args.n_operators

14

In [18]:
import pandas as pd
# create a full data frame

# each column is defined as a seperate list
band_name = ['pantera', 'metallica', 'megadeth', 'anthrax']
n_albums = [9, 10, 15, 11]
formed = [1981, 1981, 1983, 1981]
still_active = [0, 1, 1, 1]

# empty dataframe
thrash_metal_bands = pd.DataFrame()

# create new columns from lists
thrash_metal_bands['band'] = band_name
thrash_metal_bands['n_albums'] = n_albums
thrash_metal_bands['yr_formed'] = formed
thrash_metal_bands['active'] = still_active

thrash_metal_bands

,band,n_albums,yr_formed,active
0,pantera,9,1981,0
1,metallica,10,1981,1
2,megadeth,15,1983,1
3,anthrax,11,1981,1


In [24]:
thrash_metal_bands[thrash_metal_bands.columns[1:]].T.to_dict()

{0: {'n_albums': 9, 'yr_formed': 1981, 'active': 0},
 1: {'n_albums': 10, 'yr_formed': 1981, 'active': 1},
 2: {'n_albums': 15, 'yr_formed': 1983, 'active': 1},
 3: {'n_albums': 11, 'yr_formed': 1981, 'active': 1}}

In [30]:
thrash_metal_bands[thrash_metal_bands.columns[0]].T.to_list()

['pantera', 'metallica', 'megadeth', 'anthrax']

In [39]:
def create_experiments(df_experiments):
    '''
    Returns dictionary of Experiment objects based on contents of a dataframe

    Params:
    ------
    df_experiments: pandas.DataFrame
        Dataframe of experiments. First two columns are id, name followed by 
        variable names.  No fixed width

    Returns:
    --------
    dict
    '''
    experiments = {}
    
    # experiment input parameter dictionary
    exp_dict = df_experiments[df_experiments.columns[1:]].T.to_dict()
    # names of experiments
    exp_names = df_experiments[df_experiments.columns[0]].T.to_list()
    
    # loop through params and create Experiment objects.
    for name, params in zip(exp_names, exp_dict.values()):
        experiments[name] = Experiment(**params)
    
    return experiments

In [40]:
# each column is defined as a seperate list
names = ['base', 'op+1', 'nurse+1', 'op+1_nurse+1']
operators = [13, 14, 13, 14]
nurses = [9, 9, 10, 10]


# empty dataframe
df_experiments = pd.DataFrame()

# create new columns from lists
df_experiments['experiment'] = names
df_experiments['n_operators'] = operators
df_experiments['n_nurses'] = nurses


df_experiments

,experiment,n_operators,n_nurses
0,base,13,9
1,op+1,14,9
2,nurse+1,13,10
3,op+1_nurse+1,14,10


In [41]:
exp_dict = create_experiments(df_experiments)

In [44]:
exp_dict

{'base': <model.Experiment at 0x7f7a84178e80>,
 'op+1': <model.Experiment at 0x7f7a841b7280>,
 'nurse+1': <model.Experiment at 0x7f7a841b6ca0>,
 'op+1_nurse+1': <model.Experiment at 0x7f7a841b6610>}

In [42]:
exp_dict['op+1'].n_operators

14

In [43]:
exp_dict['nurse+1'].n_operators

13

In [38]:
exp_dict['nurse+1'].n_nurses

10